# Chapter 09-04: 3D 병렬화 (3D Parallelism)

## 학습 목표
- DP, TP, PP를 **계층적으로 결합**하는 3D Parallelism의 설계 원칙과 총 GPU 수 공식 $N = D_{tp} \times D_{pp} \times D_{dp}$를 이해한다
- 각 병렬화 차원을 **어떤 하드웨어 토폴로지**에 배치해야 최적인지 네트워크 대역폭 기준으로 분석한다
- 대규모 모델(Llama-2 70B, GPT-3 175B)의 실제 3D 설정을 공식으로 역산하고 GPU 효율을 평가한다

## 목차
1. [수학적 기초: 3D 병렬화 설계 원칙](#1.-수학적-기초)
2. [하드웨어 토폴로지와 통신 매핑](#2.-하드웨어-토폴로지-매핑)
3. [3D 병렬화 GPU 배치 시각화](#3.-GPU-배치-시각화)
4. [GPU 할당 계산기 구현](#4.-GPU-할당-계산기)
5. [실제 사례 분석 및 역산](#5.-실제-사례-분석)
6. [Chapter 09 종합 정리](#6.-Chapter-09-종합정리)

## 1. 수학적 기초

### 3D 병렬화 설계 공식

$$\boxed{N_{total} = D_{tp} \times D_{pp} \times D_{dp}}$$

- $D_{tp}$: Tensor Parallelism Degree (노드 내, NVLink)
- $D_{pp}$: Pipeline Parallelism Degree (노드 간, InfiniBand)
- $D_{dp}$: Data Parallelism Degree (클러스터 레벨)

### 각 차원의 역할

| 차원 | 분할 대상 | 해결하는 문제 | 최적 위치 | 통신 빈도 |
|------|----------|------------|---------|----------|
| **TP** | 레이어 내 가중치 행렬 | 단일 레이어가 GPU에 안 올라갈 때 | 노드 내 (NVLink 600GB/s) | 매우 높음 (매 레이어) |
| **PP** | 레이어 그룹 | 모델 전체가 단일 노드에 안 올라갈 때 | 인접 노드 (IB 400Gb/s) | 중간 (스테이지 경계) |
| **DP** | 데이터 배치 | 학습 처리량 향상 | 클러스터 레벨 | 낮음 (스텝 끝) |

### 설계 원칙 계층
$$\underbrace{D_{tp} \leq \text{GPU/Node}}_\text{NVLink 대역폭 활용} \quad \Rightarrow \quad \underbrace{D_{pp} \times D_{tp} = \text{총 Stage GPU 수}}_\text{모델 전체 커버} \quad \Rightarrow \quad D_{dp} = \frac{N_{total}}{D_{tp} \times D_{pp}}$$

---

### 🐣 초등학생을 위한 친절 설명!

#### 🏢 3D 병렬화가 뭔가요?

> 회사(AI 클러스터) 안에 부서(Data Parallel 그룹)가 여러 개 있고,  
> 각 부서 안에 팀(Pipeline Stage)이 있고,  
> 각 팀 안에 팀원(Tensor Parallel GPU)이 있는 것과 같아요!
>
> - **팀원들** (TP) = 한 책상에 모여 앉아 (NVLink) 빠르게 쪽 나누어 일 처리
> - **팀들** (PP) = 같은 빌딩 내에서 (InfiniBand) 일 넘겨주며 조립 라인 구성
> - **부서들** (DP) = 서로 다른 건물에서 (이더넷) 같은 작업 복사해서 각자 수행

### 📝 연습 문제

#### 문제 1: GPU 수 분해

1024 GPU 클러스터 (노드당 8 GPU, InfiniBand 연결)에서  
70B 모델 학습 시 최적 3D 설정을 도출하라.  
(TP: 최대 8, PP: Bubble 10% 이하 목표, m=8)

<details>
<summary>💡 풀이 확인</summary>

1. **TP**: 노드 내 = 8 (NVLink 최대 활용)
2. 70B × 16B = 1120 GB → 단일 GPU(80GB × 0.7 = 56GB)에 안 들어감  
   → TP만으로: $1120 / 8 = 140$ GB > 56 GB → PP 추가 필요  
   → PP=4이면: $1120 / (8×4) = 35$ GB ≤ 56 GB ✅
3. **PP**: Bubble = $(4-1)/(8+4-1) \approx 27\%$ → m=16으로 올리면 $3/19 \approx 16\%$ OK
4. **DP**: $1024 / (8 \times 4) = 32$

최종: **TP=8, PP=4, DP=32** → 총 1024 GPU ✅
</details>

#### 문제 2: 통신 빈도 비교

동일한 1024 GPU 클러스터에서,  
TP=8 vs TP=1 설정의 노드 내 All-Reduce 횟수 차이를 설명하라.

<details>
<summary>💡 풀이 확인</summary>

24-레이어 Llama-2 모델 기준:
- **TP=8 (NVLink)**: 레이어당 2회 × 24레이어 = **48회/스텝** (빠른 NVLink 활용)
- **TP=1**: 노드 내 All-Reduce 없음, 대신 DP All-Reduce가 IB로 처리

→ TP All-Reduce는 고속 NVLink에서 처리하므로 오버헤드가 작아
TP=8이 메모리와 속도 양면에서 유리
</details>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

np.random.seed(42)
print("라이브러리 로드 완료")

## 2. 하드웨어 토폴로지 매핑

In [ ]:
# ---------------------------------------------------
# 하드웨어 토폴로지와 통신 대역폭 분석
# ---------------------------------------------------

# 실제 H100 DGX 노드 기준 대역폭
bandwidth = {
    'NVLink (노드 내)':        600,   # GB/s
    'NVSwitch (노드 내)':       900,   # GB/s
    'InfiniBand HDR (노드 간)':  50,   # GB/s  (400 Gb/s)
    'InfiniBand NDR (노드 간)': 100,   # GB/s  (800 Gb/s)
    'Ethernet 100GbE':          12.5,  # GB/s
}

print("=== 하드웨어 통신 대역폭 (H100 DGX 기준) ===")
for name, bw in bandwidth.items():
    bar = '█' * int(bw / 30)
    print(f"{name:<30}: {bw:>5.0f} GB/s  {bar}")

print("\n=== 병렬화 차원별 최적 배치 원칙 ===")
print("  [TP] 노드 내 NVLink 전용  → 매 레이어 All-Reduce, 대역폭 최우선")
print("  [PP] 인접 노드 InfiniBand → 스테이지 경계 P2P, 지연 최소화 필요")
print("  [DP] 무관 노드 InfiniBand → 스텝 끝 All-Reduce, 대역폭 중요")

# 모델 크기별 필요 통신량 계산
print("\n=== GPT-3 175B 학습 시 통신량 분석 (스텝 당) ===")
P = 175e9
d_model = 12288
n_layers = 96
tp = 8; pp = 8; dp = 64

# TP: 레이어당 2회 × n_layers (활성화 크기 기준)
tp_comm_per_step = 2 * 2 * n_layers * 2  # FP16, 왕복
# PP: 스테이지 경계에서 activaton 전달
pp_comm = d_model * 2048 * 2  # seq_len=2048, FP16
# DP: 전체 그래디언트 All-Reduce
dp_comm = P * 2 * 2  # FP16, 왕복

print(f"  TP All-Reduce (NVLink):  {tp_comm_per_step/1e6:>8.1f} MB/layer × {n_layers} layers")
print(f"  PP P2P (InfiniBand):     {pp_comm/1e6:>8.1f} MB/stage boundary")
print(f"  DP All-Reduce (IB):      {dp_comm/1e9:>8.1f} GB/step (전체 그래디언트)")

## 3. GPU 배치 시각화

In [ ]:
# ---------------------------------------------------
# 3D Parallelism GPU 배치 시각화
# TP=4, PP=2, DP=2 → 총 16 GPU 예시
# ---------------------------------------------------

fig, ax = plt.subplots(figsize=(15, 9))
ax.set_xlim(0, 15)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('3D Parallelism: TP=4 (NVLink) | PP=2 (InfiniBand) | DP=2\n총 GPU = 4 × 2 × 2 = 16',
             fontsize=13, fontweight='bold', pad=15)

def draw_gpu_box(ax, x, y, gpu_id, stage, tp_idx, dp_rank, color):
    rect = FancyBboxPatch((x, y), 2.2, 1.1, boxstyle='round,pad=0.1',
                          facecolor=color, edgecolor='white', linewidth=2, alpha=0.85)
    ax.add_patch(rect)
    ax.text(x+1.1, y+0.75, f'GPU {gpu_id}', ha='center', fontsize=9, fontweight='bold', color='white')
    ax.text(x+1.1, y+0.3,  f'TP{tp_idx}|PP{stage}|DP{dp_rank}',
            ha='center', fontsize=7, color='white', alpha=0.9)

tp_colors_dp0 = ['#1565C0', '#1976D2', '#1E88E5', '#42A5F5']  # DP Replica 0 (파란색 계열)
tp_colors_dp1 = ['#B71C1C', '#C62828', '#D32F2F', '#EF5350']  # DP Replica 1 (빨간색 계열)

gpu_id = 0
for dp in range(2):
    x_base = 7.5 * dp + 0.3
    colors = tp_colors_dp0 if dp == 0 else tp_colors_dp1
    
    # DP 그룹 배경
    dp_bg = FancyBboxPatch((x_base - 0.3, 0.3), 7.2, 9,
                           boxstyle='round,pad=0.2',
                           facecolor='#E3F2FD' if dp==0 else '#FFF3E0',
                           edgecolor='#1565C0' if dp==0 else '#E65100',
                           linewidth=2.5, alpha=0.4)
    ax.add_patch(dp_bg)
    ax.text(x_base + 3.3, 9.0, f'DP Replica {dp}',
            ha='center', fontsize=11, fontweight='bold',
            color='#1565C0' if dp==0 else '#E65100')

    for pp in range(2):
        y_base = 1.5 + (1 - pp) * 4

        # PP Stage 배경
        pp_bg = FancyBboxPatch((x_base - 0.15, y_base - 0.2), 6.8, 3.2,
                               boxstyle='round,pad=0.1',
                               facecolor='#E8EAF6' if pp==0 else '#E8F5E9',
                               edgecolor='#5C6BC0' if pp==0 else '#43A047',
                               linewidth=1.5, alpha=0.6)
        ax.add_patch(pp_bg)
        ax.text(x_base + 3.2, y_base + 2.8,
                f'PP Stage {pp} (Layer {pp*12}~{pp*12+11})',
                ha='center', fontsize=9, fontweight='bold',
                color='#5C6BC0' if pp==0 else '#43A047')

        for tp in range(4):
            x_gpu = x_base + tp * 1.6
            y_gpu = y_base + 0.5 if pp==1 else y_base + 1.0
            draw_gpu_box(ax, x_gpu, y_gpu, gpu_id, pp, tp, dp, colors[tp])
            gpu_id += 1

# 범례
legend_items = [
    (mpatches.Patch(facecolor='#42A5F5', label='TP: NVLink (600 GB/s, 고속)')),
    (mpatches.Patch(facecolor='#66BB6A', label='PP: InfiniBand P2P (50 GB/s, 중간)')),
    (mpatches.Patch(facecolor='#FFA726', label='DP: InfiniBand All-Reduce (50 GB/s, 저빈도)')),
]
ax.legend(handles=legend_items, loc='lower center', fontsize=9,
          ncol=3, bbox_to_anchor=(0.5, -0.02))

plt.tight_layout()
plt.show()

print(f"TP=4, PP=2, DP=2 → {4*2*2} GPU 사용")
print("배치 원칙: TP(파란계열)은 동일 노드 내, PP(Stage 색)는 노드 간, DP(Replica)는 클러스터 간")

## 4. GPU 할당 계산기

In [ ]:
# ---------------------------------------------------
# 3D Parallelism 설정 계산기
# ---------------------------------------------------

def recommend_3d_config(model_params_B, gpu_vram_GB=80, gpus_per_node=8,
                        total_gpus=None, target_bubble_pct=10.0):
    """
    모델 크기에 따른 최적 3D 병렬화 설정 권장.
    
    전략:
    1. TP: NVLink 범위(gpus_per_node) 내에서 모델이 단일 GPU에 올라갈 때까지 증가
    2. PP: TP로도 부족하면 파이프라인 추가
    3. DP: 남는 GPU 수로 데이터 병렬화
    """
    P = model_params_B
    total_mem_GB = P * 16  # Mixed Precision + Adam: 16P bytes
    usable_GB    = gpu_vram_GB * 0.7  # 70% 활용 (activation 등 여유)

    # ── Step 1: TP 결정 (노드 내 범위) ─────────────────
    tp = 1
    while total_mem_GB / tp > usable_GB and tp < gpus_per_node:
        tp *= 2

    # ── Step 2: PP 결정 (노드 간 범위) ─────────────────
    pp = 1
    while total_mem_GB / (tp * pp) > usable_GB:
        pp *= 2

    # ── Step 3: 최소 마이크로배치 수 (Bubble 목표) ──────
    m = 1
    while (pp - 1) / (m + pp - 1) > target_bubble_pct / 100:
        m += 1

    # ── Step 4: DP 결정 ─────────────────────────────────
    min_gpus = tp * pp
    if total_gpus is None:
        dp = 1
        total = min_gpus
    else:
        dp = max(1, total_gpus // min_gpus)
        total = tp * pp * dp

    bubble = (pp - 1) / (m + pp - 1) * 100
    mem_per_gpu = total_mem_GB / (tp * pp)

    print(f"\n{'═'*60}")
    print(f"  모델: {model_params_B}B params")
    print(f"{'═'*60}")
    print(f"  학습 메모리 (16P):        {total_mem_GB:>8.1f} GB")
    print(f"  GPU당 사용 가능:          {usable_GB:>8.1f} GB")
    print(f"")
    print(f"  ┌─ 권장 3D 설정 ──────────────────────────┐")
    print(f"  │  Tensor Parallel (TP):  {tp:>4}             │")
    print(f"  │  Pipeline Parallel (PP):{pp:>4}             │")
    print(f"  │  Data Parallel (DP):    {dp:>4}             │")
    print(f"  │  총 GPU 수: {tp}×{pp}×{dp} = {total}        │")
    print(f"  └─────────────────────────────────────────┘")
    print(f"  GPU당 모델 메모리:  {mem_per_gpu:>6.1f} / {gpu_vram_GB} GB")
    print(f"  최소 마이크로배치:  m ≥ {m} (Bubble ≤ {target_bubble_pct:.0f}%)")
    print(f"  실제 Bubble 비율:   {bubble:.1f}%")

    return {'tp': tp, 'pp': pp, 'dp': dp, 'total': total, 'm_min': m}


# 다양한 모델 크기 테스트
for size in [7, 13, 70, 175, 530]:
    recommend_3d_config(size, total_gpus=1024)

## 5. 실제 사례 분석

In [ ]:
# ---------------------------------------------------
# 실제 대규모 학습 사례 분석 및 시각화
# ---------------------------------------------------

real_configs = [
    # (이름,  크기B, TP, PP, DP, m,     GPU/node, 총GPU, 참고)
    ('GPT-3',           175,  8,  8,  64, 32,  8,  4096, 'A100 80GB'),
    ('PaLM 540B',       540,  8, 16,  24,  8,  8,  3072, 'TPUv4'),
    ('Llama-2 70B',      70,  8,  4,  16, 16,  8,   512, 'A100 80GB'),
    ('Megatron-Turing', 530,  8, 35,   6,  8,  8,  4480, 'A100 80GB'),
    ('DeepSeek-V3',     671,  8, 16, 128, 'L',  8, 16384, 'H800 80GB'),
]

print(f"{'모델':<20} {'크기':>6} | {'TP':>3} | {'PP':>4} | {'DP':>5} | {'m':>5} | {'총GPU':>6} | {'Bubble':>7} | GPU/node")
print("-" * 90)
for name, size, tp, pp, dp, m, gpn, ngpu, hw in real_configs:
    if isinstance(m, int):
        bub = f"{(pp-1)/(m+pp-1)*100:.0f}%"
    else:
        bub = 'large'
    print(f"{name:<20} {size:>5}B | {tp:>3} | {pp:>4} | {dp:>5} | {str(m):>5} | {ngpu:>6} | {bub:>7} | {hw}")

print("\n공통 패턴:")
print("  ✅ TP ≤ 8 (노드당 GPU 수): NVLink 범위 내 유지")
print("  ✅ PP × TP = 모델을 완전히 커버")
print("  ✅ DP = 나머지 → 처리량(throughput) 향상목적")

In [ ]:
# ---------------------------------------------------
# 3D 설정별 이론 학습 처리량 비교
# ---------------------------------------------------

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: Bubble 비율 vs PP Degree
ax1 = axes[0]
pp_range = np.arange(1, 33)
for m_val, c in [(4, '#1E88E5'), (8, '#43A047'), (16, '#FB8C00'), (32, '#E53935')]:
    bubbles = [(pp - 1) / (m_val + pp - 1) * 100 for pp in pp_range]
    ax1.plot(pp_range, bubbles, '-o', color=c, lw=2, ms=4, label=f'm={m_val}')

ax1.axhline(y=10, color='k', ls='--', lw=1.5, label='10% 목표')
ax1.set_xlabel('Pipeline Stage 수 (PP)', fontsize=11)
ax1.set_ylabel('Bubble 비율 (%)', fontsize=11)
ax1.set_title('PP Degree별 Bubble 비율\n(마이크로배치 수 m에 따라)', fontweight='bold')
ax1.legend(fontsize=9); ax1.grid(True, alpha=0.3)

# 오른쪽: TP vs 메모리 절약
ax2 = axes[1]
tp_vals = [1, 2, 4, 8]
models_mem = {
    'Llama-2 7B':  7*16,
    'Llama-2 70B': 70*16,
    'GPT-3 175B':  175*16,
}
gpu_vram = 80  # GB

x = np.arange(len(tp_vals))
w = 0.25
colors_bm = ['#42A5F5', '#EF5350', '#66BB6A']

for i, (name, mem) in enumerate(models_mem.items()):
    mem_per_gpu = [mem / tp for tp in tp_vals]
    ax2.bar(x + i*w, mem_per_gpu, w, label=name, color=colors_bm[i], alpha=0.85)

ax2.axhline(y=gpu_vram * 0.7, color='k', ls='--', lw=2, label=f'H100 56GB (70%)')
ax2.set_xticks(x + w)
ax2.set_xticklabels([f'TP={t}' for t in tp_vals])
ax2.set_ylabel('GPU당 모델 메모리 (GB)', fontsize=11)
ax2.set_yscale('log')
ax2.set_title('TP Degree별 GPU당 모델 메모리\n(점선 아래가 H100 수용 가능)', fontweight='bold')
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 6. Chapter 09 종합정리

### 전체 병렬화 전략 비교

| 전략 | 분할 대상 | 해결 문제 | 통신 | 통신 빈도 |
|------|----------|----------|------|----------|
| **DP** | 데이터 배치 | 처리량 향상 | All-Reduce (그래디언트) | 스텝 끝 1회 |
| **TP** | 레이어 내 가중치 | 단일 레이어 메모리 | All-Reduce (활성화) | 매 레이어 2배 |
| **PP** | 레이어 그룹 | 모델 전체 메모리 | P2P (활성화 배치) | 스테이지 경계 |

### Chapter 09 핵심 공식 총정리

| 공식 | 설명 |
|------|------|
| $M_{training} = 16P$ bytes | Mixed Precision + Adam 학습 메모리 |
| $T_{Ring} \approx 2K$ | Ring All-Reduce 통신 비용 (상수 수렴) |
| $\text{Bubble} = \frac{p-1}{m+p-1}$ | GPipe/1F1B Bubble 비율 |
| $N_{total} = D_{tp} \times D_{pp} \times D_{dp}$ | 3D 병렬화 총 GPU 수 |
| $D_{tp} \leq \text{GPU/Node}$ | TP는 NVLink 범위 내 배치 |

### 다음 챕터 예고
**Chapter 10: 메모리 최적화 (ZeRO & Gradient Checkpointing)** — 분산 병렬화와는 별개로, _단일 학습 단계_에서 GPU 메모리를 극단적으로 절약하는 기법들을 다룬다. ZeRO Stage 1/2/3의 단계별 파티셔닝 수식, Gradient Checkpointing의 연산-메모리 Trade-off를 학습한다.